In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from  sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from  sklearn.metrics import *
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn import model_selection
import plotly.express as px
import rfpimp

/Users/Anti/.pyenv/versions/3.6.10/envs/data-science/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.



In [2]:
df = pd.read_csv("../data/raw/bank-additional-full.csv", sep=";")

Link to Dataset: https://www.kaggle.com/volodymyrgavrysh/bank-marketing-campaigns-dataset
It is helpful to know that for all missing categorical values the string "unknown" is used.

In [3]:
df.shape

(41188, 21)

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [5]:
list(df.columns)

['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'y']

In [6]:
df.rename(columns={"emp.var.rate":"emp_var_rate", "cons.price.idx":"cons_price_idx", "cons.conf.idx":"cons_conf_idx", "nr.employed":"nr_employed"}, inplace=True)

In [7]:
df[df.age == "unknown"]

/Users/Anti/.pyenv/versions/3.6.10/envs/data-science/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y


In [8]:
df.age = df.age.astype("uint8")

No unknown values in the age column.

In [9]:
df.job.value_counts()

admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
unknown            330
Name: job, dtype: int64

In [10]:
df["job"].replace({"admin.":"admin"}, inplace=True)

I replaces "admin." with admin because I see no reason for the dot.

Let's have a look on the marital column.

In [11]:
df.marital.value_counts()

married     24928
single      11568
divorced     4612
unknown        80
Name: marital, dtype: int64

Next I gonna have a look on the education column.

In [12]:
df.education.value_counts()

university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: education, dtype: int64

Next the default column.

In [13]:
df.default.value_counts()

no         32588
unknown     8597
yes            3
Name: default, dtype: int64

Column housing:

In [14]:
df.housing.value_counts()

yes        21576
no         18622
unknown      990
Name: housing, dtype: int64

Column loan:

In [15]:
df.loan.value_counts()

no         33950
yes         6248
unknown      990
Name: loan, dtype: int64

Column contact:

In [16]:
df.contact.value_counts()

cellular     26144
telephone    15044
Name: contact, dtype: int64

Column month:

In [17]:
df.month.value_counts()

may    13769
jul     7174
aug     6178
jun     5318
nov     4101
apr     2632
oct      718
sep      570
mar      546
dec      182
Name: month, dtype: int64

Column day_of_week

In [18]:
df.day_of_week.value_counts()

thu    8623
mon    8514
wed    8134
tue    8090
fri    7827
Name: day_of_week, dtype: int64

Column campaign:

In [19]:
df.campaign.value_counts()

1     17642
2     10570
3      5341
4      2651
5      1599
6       979
7       629
8       400
9       283
10      225
11      177
12      125
13       92
14       69
17       58
15       51
16       51
18       33
20       30
19       26
21       24
22       17
23       16
24       15
27       11
29       10
25        8
26        8
28        8
30        7
31        7
35        5
33        4
32        4
34        3
40        2
42        2
43        2
37        1
39        1
41        1
56        1
Name: campaign, dtype: int64

In [20]:
df.campaign = df.campaign.astype("uint8")

Column pdays:

In [21]:
df.pdays.value_counts(ascending=True)

27         1
25         1
20         1
26         1
21         2
22         3
19         3
18         7
17         8
16        11
0         15
8         18
14        20
15        24
1         26
11        28
13        36
5         46
10        52
12        58
7         60
2         61
9         64
4        118
6        412
3        439
999    39673
Name: pdays, dtype: int64

In [22]:
df.pdays = df.pdays.astype("uint8")

Column previous:

In [23]:
df.previous.value_counts()

0    35563
1     4561
2      754
3      216
4       70
5       18
6        5
7        1
Name: previous, dtype: int64

In [24]:
df.previous = df.previous.astype("uint8")

Cloumn poutcome:

In [25]:
df.poutcome.value_counts()

nonexistent    35563
failure         4252
success         1373
Name: poutcome, dtype: int64

Column emp_var_rate

In [26]:
df.emp_var_rate.value_counts()

 1.4    16234
-1.8     9184
 1.1     7763
-0.1     3683
-2.9     1663
-3.4     1071
-1.7      773
-1.1      635
-3.0      172
-0.2       10
Name: emp_var_rate, dtype: int64

Column cons_price_inx:

In [27]:
df.cons_price_idx.value_counts()

93.994    7763
93.918    6685
92.893    5794
93.444    5175
94.465    4374
93.200    3616
93.075    2458
92.201     770
92.963     715
92.431     447
92.649     357
94.215     311
94.199     303
92.843     282
92.379     267
93.369     264
94.027     233
94.055     229
93.876     212
94.601     204
92.469     178
93.749     174
92.713     172
94.767     128
93.798      67
92.756      10
Name: cons_price_idx, dtype: int64

Column cons_conf_idx:

In [28]:
df.cons_conf_idx.value_counts()

-36.4    7763
-42.7    6685
-46.2    5794
-36.1    5175
-41.8    4374
-42.0    3616
-47.1    2458
-31.4     770
-40.8     715
-26.9     447
-30.1     357
-40.3     311
-37.5     303
-50.0     282
-29.8     267
-34.8     264
-38.3     233
-39.8     229
-40.0     212
-49.5     204
-33.6     178
-34.6     174
-33.0     172
-50.8     128
-40.4      67
-45.9      10
Name: cons_conf_idx, dtype: int64

Column euribor3m:

In [29]:
df.euribor3m.value_counts()

4.857    2868
4.962    2613
4.963    2487
4.961    1902
4.856    1210
         ... 
1.045       1
0.956       1
0.933       1
3.282       1
0.996       1
Name: euribor3m, Length: 316, dtype: int64

Column nr_employeed:

In [30]:
df.nr_employed.value_counts()

5228.1    16234
5099.1     8534
5191.0     7763
5195.8     3683
5076.2     1663
5017.5     1071
4991.6      773
5008.7      650
4963.6      635
5023.5      172
5176.3       10
Name: nr_employed, dtype: int64

Column y:

In [31]:
df.y.value_counts()

no     36548
yes     4640
Name: y, dtype: int64

In [32]:
df.y.replace({"yes":1, "no":0}, inplace=True)
df.y = df.y.astype("uint8")

In [33]:
df.isna().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp_var_rate      0
cons_price_idx    0
cons_conf_idx     0
euribor3m         0
nr_employed       0
y                 0
dtype: int64

In [34]:
df.shape

(41188, 21)

In [35]:
df.drop_duplicates(inplace=True)

In [36]:
df.shape

(41176, 21)

In [37]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp_var_rate', 'cons_price_idx',
       'cons_conf_idx', 'euribor3m', 'nr_employed', 'y'],
      dtype='object')

In [38]:
df.to_csv("../data/clean/clean.csv", index=False)

Column duration:

In [39]:
df.duration.value_counts()

85      170
90      170
136     168
73      167
124     163
       ... 
1108      1
980       1
4918      1
2453      1
2015      1
Name: duration, Length: 1544, dtype: int64

In [40]:
df.duration = df.duration.astype("uint8")

In [41]:
df.drop("duration", axis=1, inplace=True)

In [42]:
df.shape

(41176, 20)

As written in the explaination on kaggle I will drop the duration column because the duration cannot be known in advance but after a long talk it is quite sure that the client will create an account.

Further optimization for modeling.

In [43]:
df.emp_var_rate = df.emp_var_rate * 10
df.emp_var_rate = df.emp_var_rate.astype("int8")

In [44]:
df.cons_conf_idx = df.cons_conf_idx * (-10)
df.cons_conf_idx = df.cons_conf_idx.astype("uint16")

In [45]:
df.nr_employed = df.nr_employed * 10
df.nr_employed = df.nr_employed.astype("uint16")

One Hot encoding

In [46]:
categorical_columns = ["job", "marital", "education", "default", "housing", "loan", "contact", "month", "day_of_week", "poutcome"]

In [47]:
for col in categorical_columns:
    df[col] = df[col].astype('category')

In [48]:
df = pd.get_dummies(df, columns=categorical_columns, drop_first=False)

In [49]:
df.shape

(41176, 63)

In [50]:
df.columns

Index(['age', 'campaign', 'pdays', 'previous', 'emp_var_rate',
       'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'nr_employed', 'y',
       'job_admin', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'marital_unknown', 'education_basic.4y', 'education_basic.6y',
       'education_basic.9y', 'education_high.school', 'education_illiterate',
       'education_professional.course', 'education_university.degree',
       'education_unknown', 'default_no', 'default_unknown', 'default_yes',
       'housing_no', 'housing_unknown', 'housing_yes', 'loan_no',
       'loan_unknown', 'loan_yes', 'contact_cellular', 'contact_telephone',
       'month_apr', 'month_aug', 'month_dec', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'mon

In [51]:
df.to_csv("../data/clean/clean_encoded.csv", index=False)